# Package Imports

In [1]:
import glob
import os
import numpy as np
import pandas as pd
import sqlite3 as sql
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
sns.set(font_scale=1.2, context="paper", style="ticks")
sns.set_palette("viridis")

%matplotlib inline

import plotly
plotly.offline.init_notebook_mode(connected=True)

import sys
sys.path.append("..")

In [2]:
os.environ.get("CONDA_PREFIX")

'/epyc/opt/anaconda'

In [3]:
os.nice(3)

3

In [4]:
import thor

In [2]:
# Connect to database
con = mariadb.connect(user='ztf', database='ztf')

In [ ]:
# https://zwickytransientfacility.github.io/ztf-avro-alert/schema.html

In [3]:
# Read alerts for solar system objects from after the photometry fix 
sso_alert_fix_date1 = Time('2018-05-16T23:30:00', format='isot', scale='utc') # first attribution fix
sso_alert_fix_date2 = Time('2018-06-08T23:30:00', format='isot', scale='utc') # second attribution fix
sso_alert_phot_fix_date = Time('2018-06-18T23:30:00', format='isot', scale='utc') # photometry fix date

jd_good = sso_alert_phot_fix_date.jd
#ssdistnr >= 0 
df = pd.read_sql_query('select * from alerts where jd > {} and ssdistnr >= 0'.format(jd_good), con)
print(len(df))

628667


In [ ]:
columnMapping = {        
        
        # Observation ID
        "obs_id" : "obs_id",
        
        # Exposure time
        "exp_mjd" : "exp_mjd",
        
        # Visit ID
        "visit_id" : "visit_id",
        
        # Field ID
        "field_id" : "field",
        
        # Field RA in degrees
        "field_RA_deg" : "fieldRA_deg",
        
        # Field Dec in degrees
        "field_Dec_deg" : "fieldDec_deg",
        
        # Night number
        "night": "nid",
        
        # RA in degrees
        "RA_deg" : "ra",
        
        # Dec in degrees
        "Dec_deg" : "decl",
        
        # Observer's x coordinate in AU
        "obs_x_au" : "HEclObsy_X_au",
        
        # Observer's y coordinate in AU
        "obs_y_au" : "HEclObsy_Y_au",
        
        # Observer's z coordinate in AU
        "obs_z_au" : "HEclObsy_Z_au",
        
        # Magnitude (UNUSED)
        "mag" : "magpsf",
        
        ### Truth Parameters
        
        # Object name
        "name" : "ssnamenr",
        
        # Observer-object distance in AU
        "Delta_au" : "Delta_au",
        
        # Sun-object distance in AU (heliocentric distance)
        "r_au" : "r_au",
        
        # Object's x coordinate in AU
        "obj_x_au" : "HEclObj_X_au",
        
        # Object's y coordinate in AU
        "obj_y_au" : "HEclObj_Y_au",
        
        # Object's z coordinate in AU
        "obj_z_au" : "HEclObj_Z_au",
        
        # Object's x velocity in AU per day
        "obj_dx/dt_au_p_day" : "HEclObj_dX/dt_au_p_day",
        
        # Object's y velocity in AU per day
        "obj_dy/dt_au_p_day" : "HEclObj_dY/dt_au_p_day",
        
        # Object's z velocity in AU per day
        "obj_dz/dt_au_p_day" : "HEclObj_dZ/dt_au_p_day",
        
        # Semi-major axis
        "a_au" : "a_au",
        
        # Inclination
        "i_deg" : "i_deg",
        
        # Eccentricity
        "e" : "e",
    }

def prepareZTFAlertsForTHOR(alerts):
    # Sort by field and exposure time
    alerts.sort_values(by=["field", "jd"], inplace=True)
    
    # Add exposure time in MJD column
    times = Time(alerts["jd"], format="jd", scale="utc")
    times = times.mjd
    alerts["exp_mjd"] = times
    
    # Only keep columns we want
    alerts = alerts[["ra", "decl", "field", "nid", "jd", "exp_mjd", "magpsf", "sigmapsf", "fid", "ssnamenr"]]
    
    # Read footprint and get field center locations
    footprint = pd.read_csv("/epyc/projects/rascals/ztf_information/field_grid/ZTF_Fields.txt",
                        delim_whitespace=True, 
                        skiprows=1,
                        names=["field", "fieldRA_deg", "fieldDec_deg", "Ebv", "Gal Long", "Gal Lat", "Ecl Long", "Ecl Lat" ,"Entry"])
    footprint = footprint[["field", "fieldRA_deg", "fieldDec_deg"]]
    
    os.environ["OORB_DATA"] = "/astro/users/moeyensj/.conda/envs/python3-moeyensj-thor/share/openorb"
    print(os.environ["OORB_DATA"])

    survey = pd.DataFrame({"exp_mjd": alerts["exp_mjd"].unique(), "visit_id" : np.arange(1, len(alerts["exp_mjd"].unique()) + 1)})
    eph = propagateTestParticle([1, 1, 0, 0.0002, 0.0002, 0.000], 59580, survey["exp_mjd"].values, observatoryCode="I41")
    survey = survey.merge(eph[["mjd", 'HEclObsy_X_au', 'HEclObsy_Y_au', 'HEclObsy_Z_au']], left_on="exp_mjd", right_on="mjd")
    
    observations = alerts.merge(footprint, left_on="field", right_on="field")
    observations = observations.merge(survey, on="exp_mjd")

    observations["obs_id"] = np.arange(1, len(observations) + 1)
    
    return observations

In [ ]:
observations = prepareZTFAlertsForTHOR(df)

In [ ]:
observations.to_csv("../analysis/ztf/observations.txt", sep=" ", index=False)

# Load Data

In [ ]:
observations = pd.read_csv("../analysis/ztf/observations.txt", index_col=False, sep=" ", low_memory=False)

In [15]:
average_orbits = thor.findAverageOrbits(observations[(observations["night"] == 59740)], rValues=[0.0, 2.5, 5.0, 10.0])

THOR: findAverageObject
-------------------------
Finding average orbit in 3 heliocentric distance bins...
Done.
-------------------------



In [16]:
average_orbits

,orbit_id,r_au,HEclObj_dX/dt_au_p_day,HEclObj_dY/dt_au_p_day,HEclObj_dZ/dt_au_p_day,exp_mjd,RA_deg,Dec_deg,designation
0,1,2.314816,0.004529,-0.010996,-0.000038,59740.246528,176.340608,5.265090,K14J09X
1,2,2.759569,0.002326,-0.009072,-0.000009,59740.247222,175.959376,-0.407342,b3451
2,3,5.143260,0.000385,-0.006312,0.000082,59740.219444,168.999899,0.675941,m8613


In [ ]:
allObjects_survey, summary_survey, summary_orbits = thor.runTHOR(observations, 
                                                                average_orbits, 
                                                                knownOrbits=orbits,
                                                                runDir="../analysis/msst_4x4/run_1/", 
                                                                cellArea=100,
                                                                cellShape="circle",
                                                                threads=20,
                                                                vxBins=300,
                                                                vyBins=300,
                                                                eps=0.005,
                                                                minSamples=5)

THOR: runTHOR
-------------------------
Running THOR with 3 orbits...

THOR: analyzeObservations
-------------------------
Analyzing observations...
Known object observations: 160724
Unknown object observations: 0
False positive observations: 127468
Percent known object observations (%): 55.770
Percent unknown object observations (%): 0.000
Percent false positive observations (%): 44.230
Unique known objects: 25628
Unique known objects with at least 5 detections: 19234

Total time in seconds: 0.7677481174468994
-------------------------

THOR: runTHOR
-------------------------
Running orbit 1...

THOR: analyzeProjections
-------------------------
Analyzing projections...
Known object observations: 75645
Unknown object observations: 0
False positive observations: 56723
Percent known object observations (%): 57.147
Percent unknown object observations (%): 0.000
Percent false positive observations (%): 42.853
Unique known objects: 12105
Unique known objects with at least 5 detections: 974